# 为互联网电影评论数据，包含50000个关于电影的正面/负面评论，请采用逻辑回归模型或SVM或其他机器学习模型来拟合数据，并通过交叉验证法评估模型效果。
提示：可以参考网友做法，详见https://www.jianshu.com/p/c9165bfa7619?utm_campaign=maleskine&utm_content=note&utm_medium=seo_notes&utm_source=recommendation
大致思路：读取数据、清洗文本、提取词汇、移除停用词、词带模型、数据集划分、逻辑回归模型拟合与评估。
需要使用python自然语言工具包nltk等。


import

In [ ]:
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt 
# import nltk
# import os
# import pyprind
# import re
# from datashader.bundling import smooth
# from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
# from sklearn.svm import SVC
# from sklearn.decomposition import LatentDirichletAllocation
# import pyLDAvis
# basepath = 'aclImdb'
# labels = {'pos': 1, 'neg': 0}
# pbar = pyprind.ProgBar(50000)
# df = pd.DataFrame()
# for s in ('test', 'train'):
#     for l in ('pos', 'neg'):
#         path=os.path.join(basepath,s,l)
#         for file in os.listdir(path):
#             with open(os.path.join(path,file),'r',encoding='utf-8') as infile:
#                 txt=infile.read()
#                 df=df._append([[txt,labels[l]]],ignore_index=True)
#                 pbar.update()
# df.columns=['review','sentiment']
# 
# np.random.seed(0)
# df=df.reindex(np.random.permutation(df.index))
# df.to_csv('movie_data.csv',encoding='utf-8')
# df.head()

In [1]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.svm import SVC
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk import PorterStemmer
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk

df=pd.read_csv('movie_data.csv',encoding='utf-8')

# nltk.download('stopwords')#停用词移除(stop-word removal)，停用词是文本中常见单不能有效判别信息的词汇  
stop = stopwords.words('english')#获得英文停用词集  

def preprocessor(text):

    text=re.sub(r'<[^>]*>','',text)
    emotions=re.findall(r'(?::|;|=)(?:-)?(?:\)|\(|D|P)',text)
    text=re.sub(r'[\W]+',' ',text.lower())+''.join(emotions).replace('-','')
    # text=''.join(porter.stem(word) for word in text.split() if word not in stop)
    return text

df['review']=df['review'].apply(preprocessor)

porter=PorterStemmer()

def tokenizer(text):#提取词汇  
    return text.split()  

def tokenizer_porter(text):#文本分词并提取词干  
    return [porter.stem(word) for word in text.split()]  

# vector=CountVectorizer(binary=True)
# vector=CountVectorizer()
#加tfidf

#50000个数据，对半分
X_train=df.loc[:25000,'review'].values  

y_train=df.loc[:25000,'sentiment'].values  

X_test=df.loc[25000:,'review'].values  

y_test=df.loc[25000:,'sentiment'].values       

# X_train=vector.fit_transform(X_train)
# X_test=vector.transform(X_test)
tfidf=TfidfVectorizer(strip_accents=None,lowercase=False,preprocessor=None)  

param_grid = [{'vect__ngram_range':[(1,1)],'vect__stop_words':[stop,None],'vect__tokenizer':[tokenizer,tokenizer_porter],'clf__penalty':['l1','l2'],'clf__C':[1.0]},{'vect__ngram_range':[(1,1)],'vect__stop_words':[stop,None],'vect__tokenizer':[tokenizer,tokenizer_porter],'vect__use_idf':[False],'vect__norm':[None],'clf__penalty':['l1','l2'],'clf__C':[1.0,100.0]} ]  

lr_tfidf =Pipeline([('vect',tfidf),('clf',LogisticRegression(random_state=0))])  

gs_lr_tfidf=GridSearchCV(lr_tfidf,param_grid,scoring='accuracy',cv=2,verbose=3,n_jobs=-1)  

gs_lr_tfidf.fit(X_train,y_train)  

print ('Best parameter set :%s' % gs_lr_tfidf.best_params_)  

print ('CV Accuracy:%.3f'%gs_lr_tfidf.best_score_)  

clf=gs_lr_tfidf.best_estimator_  

print ('Test Accuracy:%.3f'%clf.score(X_test,y_test)) 


Fitting 2 folds for each of 24 candidates, totalling 48 fits


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
24 fits failed out of a total of 48.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
24 fits failed with the following error:
Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\ProgramData\anaconda3\Lib\site-packages\sklearn\pipeline.py", line 475, in fit
    self._final_estimator.fit

Best parameter set :{'clf__C': 1.0, 'clf__penalty': 'l2', 'vect__ngram_range': (1, 1), 'vect__stop_words': ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each',

## Comments
@InProceedings{maas-EtAl:2011:ACL-HLT2011,
  author    = {Maas, Andrew L.  and  Daly, Raymond E.  and  Pham, Peter T.  and  Huang, Dan  and  Ng, Andrew Y.  and  Potts, Christopher},
  title     = {Learning Word Vectors for Sentiment Analysis},
  booktitle = {Proceedings of the 49th Annual Meeting of the Association for Computational Linguistics: Human Language Technologies},
  month     = {June},
  year      = {2011},
  address   = {Portland, Oregon, USA},
  publisher = {Association for Computational Linguistics},
  pages     = {142--150},
  url       = {http://www.aclweb.org/anthology/P11-1015}
}